In [0]:
confluentApiKey=''
confluentSecret=''

In [0]:
order_df=spark.readStream.format('kafka').option('kafka.bootstrap.servers','pkc-921jm.us-east-2.aws.confluent.cloud:9092').option('kafka.security.protocol','SASL_SSL').option('kafka.sasl.mechanism','PLAIN').option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))\
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.option("subscribe",'orders_raw_data') \
.option("startingTimestamp",1) \
.option("maxOffsetsPerTrigger",50) \
.load()

In [0]:
from pyspark.sql.functions import *
order_schema='order_id string,user_id string,product_id string,category string,quantity int,price int,payment_mode string,order_status string,event_time string'
order_df=order_df.select(col('key').cast('string').alias('key'),col('value').cast('string').alias('value'))
order_df1=order_df.select(col('key'),from_json(col('value'),order_schema).alias('order'))
order_df1=order_df1.select(col('key'),col('order.order_id').alias('order_id'),col('order.user_id').alias('user_id'),col('order.product_id').alias('product_id'),col('order.category').alias('category'),col('order.quantity').alias('quantity'),col('order.price').alias('price'),col('order.payment_mode').alias('payment_mode'),col('order.order_status').alias('order_status'),col('order.event_time').alias('event_time'))
order_df1=order_df1.withColumn('total_amount',col('price')*col('quantity'))
order_df1=order_df1.withColumn('processing_time',current_timestamp())
order_filtered_df=order_df1.filter(col('order_status')=='CREATED')
order_filtered_df=order_filtered_df.select(col('key').alias('key'),to_json(struct(col('order_id'),col('user_id'),col('product_id'),col('category'),col('quantity'),col('price'),col('payment_mode'),col('order_status'),col('event_time'),col('total_amount'),col('processing_time'))).alias('value'))

In [0]:
order_filtered_df.writeStream.queryName("ingestionquery") \
.format("kafka") \
.outputMode("append") \
.option("checkpointLocation","abfss://data@storageaccountfda.dfs.core.windows.net/catalog/retail/checkpoint1") \
.option("kafka.bootstrap.servers",'pkc-921jm.us-east-2.aws.confluent.cloud:9092') \
.option("kafka.security.protocol","SASL_SSL") \
.option("kafka.sasl.mechanism","PLAIN") \
.option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.trigger(availableNow=True)\
.option("topic",'order_enriched') \
.start()

In [0]:
order_enriched_df=spark.readStream.format('kafka').option('kafka.bootstrap.servers','pkc-921jm.us-east-2.aws.confluent.cloud:9092').option('kafka.security.protocol','SASL_SSL').option('kafka.sasl.mechanism','PLAIN').option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))\
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.option("subscribe",'order_enriched') \
.option("startingTimestamp",1) \
.option("maxOffsetsPerTrigger",50) \
.load()

In [0]:
order_enrich_schema='order_id string,user_id string,product_id string,category string,quantity int,price int,payment_mode string,order_status string,event_time timestamp,total_amount int,processing_time timestamp'
order_enriched_df1=order_enriched_df.select(col('key').cast('string').alias('key'),col('value').cast('string'))
order_enriched_df1=order_enriched_df1.select(col('key'),from_json(col('value'),order_enrich_schema).alias('value'))
order_enriched_df1=order_enriched_df1.select(col('key'),col('value.order_id').alias('order_id'),col('value.product_id').alias('product_id'),col('value.category').alias('category'),col('value.user_id').alias('user_id'),col('value.quantity').alias('quantity'),col('value.price').alias('price'),col('value.payment_mode').alias('payment_mode'),col('value.order_status').alias('order_status'),col('value.event_time').alias('event_time'),col('value.total_amount'),col('value.processing_time').alias('processing_time'))
sales_per_category_df=order_enriched_df1.groupBy(col('category'),window(col('event_time'),'5 minutes')).agg(count('*').alias('order_count'))
sales_per_category_df=sales_per_category_df.select('category',col('window.start').alias('window_start'),col('window.end').alias('window_end'),'order_count')
sales_per_category_df=sales_per_category_df.select(col('category').alias('key'),to_json(struct(col('category'),col('window_start'),col('window_end'),col('order_count'))).alias('value'))

In [0]:
sales_per_category_df.writeStream.queryName("ingestionquery") \
.format("kafka") \
.outputMode("update") \
.option("checkpointLocation","abfss://data@storageaccountfda.dfs.core.windows.net/catalog/retail/checkpoint2") \
.option("kafka.bootstrap.servers",'pkc-921jm.us-east-2.aws.confluent.cloud:9092') \
.option("kafka.security.protocol","SASL_SSL") \
.option("kafka.sasl.mechanism","PLAIN") \
.option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.trigger(availableNow=True)\
.option("topic",'order_sales') \
.start()

In [0]:
order_enriched_updated_df=spark.readStream.format('kafka').option('kafka.bootstrap.servers','pkc-921jm.us-east-2.aws.confluent.cloud:9092').option('kafka.security.protocol','SASL_SSL').option('kafka.sasl.mechanism','PLAIN').option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))\
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.option("subscribe",'order_enriched') \
.option("startingTimestamp",1) \
.option("maxOffsetsPerTrigger",50) \
.load()

In [0]:
from pyspark.sql.functions import *
order_enrich_schema='order_id string,user_id string,product_id string,category string,quantity int,price int,payment_mode string,order_status string,event_time timestamp,total_amount int,processing_time timestamp'
order_enriched_updated_df=order_enriched_updated_df.select(col('key').cast('string'),col('value').cast('string').alias('value'))
order_enriched_updated_df2=order_enriched_updated_df.select('key',from_json('value',order_enrich_schema).alias('value'))
order_enriched_updated_df2=order_enriched_updated_df2.select(col('key'),col('value.order_id').alias('order_id'),col('value.product_id').alias('product_id'),col('value.category').alias('category'),col('value.quantity').alias('quantity'),col('value.price').alias('price'),col('value.payment_mode').alias('payment_mode'),col('value.order_status').alias('order_status'),col('value.event_time').alias('event_time'),col('value.total_amount'),col('value.processing_time').alias('processing_time'))
order_enriched_updated_df2=order_enriched_updated_df2.withColumn('fraud_flag',when((col('total_amount')>50000)|((col('payment_mode')=='COD')&(col('category')=='electronics')),True).otherwise(False))
order_enriched_updated_df2=order_enriched_updated_df2.select('key',to_json(struct(col('order_id'),col('product_id'),col('category'),col('quantity'),col('price'),col('total_amount'),col('processing_time'),col('payment_mode'),col('order_status'),col('event_time'),col('fraud_flag'))).alias('value'))

In [0]:
order_enriched_updated_df2.writeStream.queryName("ingestionquery") \
.format("kafka") \
.outputMode("update") \
.option("checkpointLocation","abfss://data@storageaccountfda.dfs.core.windows.net/catalog/retail/checkpoint3") \
.option("kafka.bootstrap.servers",'pkc-921jm.us-east-2.aws.confluent.cloud:9092') \
.option("kafka.security.protocol","SASL_SSL") \
.option("kafka.sasl.mechanism","PLAIN") \
.option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.trigger(availableNow=True)\
.option("topic",'orders_flagged') \
.start()

In [0]:
user_df=spark.read.format('kafka').option('kafka.bootstrap.servers','pkc-921jm.us-east-2.aws.confluent.cloud:9092').option('kafka.security.protocol','SASL_SSL').option('kafka.sasl.mechanism','PLAIN').option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))\
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.option("subscribe",'user_events') \
.option("startingTimestamp",1) \
.option("maxOffsetsPerTrigger",50) \
.load()

In [0]:
from pyspark.sql.functions import *
user_schema='event_id string,user_id string,event_type string,page string,event_time timestamp'
user_df=user_df.select(col('key').cast('string').alias('key'),col('value').cast('string').alias('value'))
user_df1=user_df.select('key',from_json('value',user_schema).alias('value'))
user_df1=user_df1.select('key',col('value.event_id').alias('event_id'),col('value.user_id').alias('user_id'),col('value.event_type').alias('event_type'),col('value.page').alias('page'),col('value.event_time').alias('event_time'))
user_order_enrich_df=order_enriched_df1.join(user_df1,on='user_id',how='inner').drop(user_df1.user_id,order_enriched_df1.key)
user_order_enrich_df=user_order_enrich_df.select('key',to_json(struct(col('order_id'),col('product_id'),col('category'),col('quantity'),col('quantity'),col('price'),col('payment_mode'),col('order_status'),col('total_amount'),col('event_id'),col('event_type'),col('page'))).alias('value'))

In [0]:
user_order_enrich_df.writeStream.queryName("ingestionquery") \
.format("kafka") \
.outputMode("update") \
.option("checkpointLocation","abfss://data@storageaccountfda.dfs.core.windows.net/catalog/retail/checkpoint5") \
.option("kafka.bootstrap.servers",'pkc-921jm.us-east-2.aws.confluent.cloud:9092') \
.option("kafka.security.protocol","SASL_SSL") \
.option("kafka.sasl.mechanism","PLAIN") \
.option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.trigger(availableNow=True)\
.option("topic",'user_orders_join') \
.start()